In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os
path = '/content/gdrive/MyDrive/keras_retinanet'

#作業ディレクトリをpathに移動する
os.chdir(path)

In [ ]:
!pip install keras-resnet

In [ ]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
from tensorflow import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import pandas as pd
import time
import math

In [ ]:
# adjust this to point to your downloaded/trained model
# models can be downloaded here: https://github.com/fizyr/keras-retinanet/releases
model_path = 'keras_retinanet/model/resnet50_csv_100.h5'

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')

# if the model is not converted to an inference model, use the line below
# see: https://github.com/fizyr/keras-retinanet#converting-a-training-model-to-inference-model
model = models.convert_model(model)

In [ ]:
# load image
path = 'keras_retinanet/images/'
img_name = '20221129-00010000-sorae_jp-001-1-view.jpg'
image = read_image_bgr(path +img_name)

# copy to draw on
draw = image.copy()
grays = cv2.cvtColor(draw, cv2.COLOR_RGB2GRAY)
draw   = cv2.cvtColor(grays, cv2.COLOR_GRAY2RGB)

# preprocess image for network
image = preprocess_image(image)
image, scale = resize_image(image)

labels = ['rockfall']

# process image
start = time.time()
boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
print("processing time: ", time.time() - start)

# correct for image scale
boxes /= scale

# visualize detections
for box, score, label in zip(boxes[0], scores[0], labels[0]):
    # scores are sorted so we can break
    if score < 0.5:
        break
        
    color = label_color(label)
    
    b = box.astype(int)
    draw_box(draw, b, color=color)
    
    caption = "{} {:.3f}".format("crater", score)
    draw_caption(draw, b, caption,1.5,2)

plt.figure(figsize=(15, 15))
plt.axis('off')
plt.imshow(draw)
plt.savefig(path + 'equ_crater_pred_50.jpg', bbox_inches='tight',pad_inches = 0)
plt.show()